In [ ]:
%pwd
%cd /app

In [ ]:
import dac
from audiotools import AudioSignal

import torch

import numpy as np
import random
import time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display as ipd

In [ ]:
# Download a model
model_path = dac.utils.download(model_type="44khz")
model = dac.DAC.load(model_path)
model.to('cuda');

<div style="height: 10px; background-color: blue;"></div>
<span style="font-size: 24px; color:blue">Randomizing codebook entries </span>


### <font color='green'> Lets try to look at just a short excerpt of codes and the signal it would make. Dimensions are (B, Nq, T) </font>

In [ ]:
def codeMash(a, b, ttype="random", lengthfactor=12, numCodebooks=4,  verbose=0) :
    numframes=25*lengthfactor
    chunklength=60  #this seems to work for most length factors > 4
    olength=2466*lengthfactor+1

    # now make the code tensor
    if ttype=="random" :
        # a is range low, b is range high
        baz = torch.randint(low=a, high=b, size=(1, numCodebooks, numframes)) #between 0 and 1023

        #This increases code indexes with time, all codes the same at any time
    elif ttype == "timeindex"  :
        # a is start, b is step
        # Generate depth indices using torch.arange()
        depth_indices = torch.arange(numframes).unsqueeze(0).unsqueeze(1)
        # Broadcast depth indices to match the desired shape
        broadcasted_indices = depth_indices.expand(1, numCodebooks, -1)
        # Create the final 3D tensor
        baz=broadcasted_indices*b+a
    
    else :
        print(f'invalid type: {ttype}')

    # Clip to preserve reconstruction sanity
    if torch.any(baz > 1023) :
        mask = baz > 1023
        baz[mask] = 0
        print(f'clipping some codebook entries to max 1023')
    if torch.any(baz < 0) :
        mask = baz < 0
        baz[mask] = 0
        print(f'clipping some codebook entries to min  0')

    
    my_dac_file = dac.DACFile(
            codes=baz,
            chunk_length=chunklength,
            original_length=olength,
            input_db=-20,
            channels=1,
            sample_rate=44100,
            padding=False,
            dac_version='1.0.0',
        )

    if verbose>=1 :
        print(f'shape of code tensor is {baz.shape}')
    if verbose>=10 :
        print(baz)

    ydcompress = model.decompress(my_dac_file)
    ysig=ydcompress.cpu().detach().numpy()[0,0,:]
    return ysig

In [ ]:
# And this sounds like .....
# ysig = codeMash(500, 505, "random", lengthfactor=96,  numCodebooks=9, verbose=1)
#ysig = codeMash(0, 1, ttype="timeindex", lengthfactor=48 )
ysig=codeMash(0, 1023, lengthfactor=100, numCodebooks=4)

plt.plot(ysig)
ipd.Audio(ysig, rate=44100)

In [ ]:
# So lets string a bunch of them together.....

sbuf=[]
sdur=1 # sleep time in seconds

start_time = time.time()
for x in range(8,30):
    
    #SAVE ysig=codeMash(1010-10*x, 1013-10*x)
    #SAVE ysig=codeMash(1010-4*x, 1013-3*x)
    ysig=codeMash(1000-x, 1003-x)

    # Can't get this to play in the loop.....
    # ipd.Audio(audio_segment, rate=44100, autoplay=True)
    # time.sleep(sdur+.1)
    
    sbuf.extend(ysig)
end_time = time.time()

print(f"Codebook and audio generation time: {end_time - start_time} seconds")
print(f'sbuf.len={len(sbuf)}, sound is {len(sbuf)/44100} seconds long', flush=True)

plt.plot(ysig)
ipd.Audio(sbuf, rate=44100)

<div style="height: 10px; background-color: blue;"></div>